In [85]:
import pandas as pd
import numpy as np
import json
import re

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.decomposition import PCA, TruncatedSVD

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from nltk import ngrams, everygrams

from catboost import CatBoostClassifier, cv

### Data

In [2]:
train = pd.read_json('train.json').set_index('id')

In [3]:
train

,cuisine,ingredients
id,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
22213,indian,"[water, vegetable oil, wheat, salt]"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...
29109,irish,"[light brown sugar, granulated sugar, butter, ..."
11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b..."
2238,irish,"[eggs, citrus fruit, raisins, sourdough starte..."


In [4]:
test = pd.read_json('test.json').set_index('id')

In [5]:
test

,ingredients
id,
18009,"[baking powder, eggs, all-purpose flour, raisi..."
28583,"[sugar, egg yolks, corn starch, cream of tarta..."
41580,"[sausage links, fennel bulb, fronds, olive oil..."
29752,"[meat cuts, file powder, smoked sausage, okra,..."
35687,"[ground black pepper, salt, sausage casings, l..."
...,...
30246,"[large egg yolks, fresh lemon juice, sugar, bo..."
36028,"[hot sauce, butter, sweet potatoes, adobo sauc..."
22339,"[black pepper, salt, parmigiano reggiano chees..."


In [6]:
with open('sample_submission.csv') as f:
    sample_submission = f.readlines()

In [7]:
len(sample_submission)

9945

In [8]:
sample_submission[:5]

['id,cuisine\n',
 '35203,italian\n',
 '17600,italian\n',
 '35200,italian\n',
 '17602,italian\n']

### Target: cuisine

In [8]:
len(train.cuisine.unique())

20

The sample is imbalanced:

In [9]:
train.cuisine.value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [10]:
train.isna().sum()

cuisine        0
ingredients    0
dtype: int64

In [11]:
train.all(axis=0)

cuisine        True
ingredients    True
dtype: bool

### Feature engineering

In [12]:
def get_ngrams(l: list, n: int):
    ngrams_list = []
    for item in l:
        grams = list(everygrams(item, max_len=n))
        ngrams_list += grams
    ngrams_list1 = [''.join(i) for i in ngrams_list]
    return set(ngrams_list1)

In [56]:
train['ngrams'] = train['ingredients'].apply(lambda x: get_ngrams(x, 3))
test['ngrams'] = test['ingredients'].apply(lambda x: get_ngrams(x, 3))

In [57]:
ngrams_dict_train = train[['ngrams']].to_dict(orient='records')
ngrams_dict_test = test[['ngrams']].to_dict(orient='records')

In [58]:
vectorizer = DictVectorizer()

In [59]:
X = vectorizer.fit_transform(ngrams_dict_train)
X_kaggle = vectorizer.transform(ngrams_dict_test)

In [60]:
print(X.shape)
print(X_kaggle.shape)

(39774, 6319)
(9944, 6319)


In [61]:
X_df = pd.DataFrame(X.toarray(), index=train.index, columns=vectorizer.get_feature_names_out())
X_df.head()

,ngrams=,ngrams=,ngrams=,ngrams= b,ngrams= c,ngrams= o,ngrams= &,ngrams= &,ngrams= ',ngrams= 'N,...,ngrams=™ -,ngrams=™ C,ngrams=™ R,ngrams=™ T,ngrams=™ c,ngrams=™ m,ngrams=™ o,ngrams=™ r,ngrams=™ s,ngrams=™ t
id,,,,,,,,,,,,,,,,,,,,,
10259,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25693,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
20130,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
22213,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13162,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
y = train['cuisine']
y

id
10259          greek
25693    southern_us
20130       filipino
22213         indian
13162         indian
            ...     
29109          irish
11462        italian
2238           irish
41882        chinese
2362         mexican
Name: cuisine, Length: 39774, dtype: object

### Logistic Regression

In [67]:
model = LogisticRegression(multi_class='ovr', random_state=10, max_iter=1000)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=6000, random_state=10)

In [69]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, multi_class='ovr', random_state=10)

In [70]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [71]:
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.9001302777284301
0.7806666666666666


The model overfits.

In [72]:
cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

0.7819933031018869

Cross-validation score: 0.782

### CatBoost

In [90]:
cb = CatBoostClassifier(objective='MultiClass', task_type='GPU', random_state=10)

In [91]:
cb.fit(X_train, y_train)

Learning rate set to 0.133081
0:	learn: 2.4960336	total: 93.8ms	remaining: 1m 33s
1:	learn: 2.3039577	total: 192ms	remaining: 1m 35s
2:	learn: 2.1780611	total: 286ms	remaining: 1m 35s
3:	learn: 2.0794634	total: 384ms	remaining: 1m 35s
4:	learn: 2.0023037	total: 484ms	remaining: 1m 36s
5:	learn: 1.9332647	total: 569ms	remaining: 1m 34s
6:	learn: 1.8793365	total: 655ms	remaining: 1m 32s
7:	learn: 1.8370572	total: 729ms	remaining: 1m 30s
8:	learn: 1.7901578	total: 811ms	remaining: 1m 29s
9:	learn: 1.7485892	total: 899ms	remaining: 1m 28s
10:	learn: 1.7202112	total: 968ms	remaining: 1m 26s
11:	learn: 1.6946853	total: 1.04s	remaining: 1m 25s
12:	learn: 1.6680698	total: 1.12s	remaining: 1m 24s
13:	learn: 1.6434803	total: 1.2s	remaining: 1m 24s
14:	learn: 1.6145892	total: 1.27s	remaining: 1m 23s
15:	learn: 1.5942070	total: 1.34s	remaining: 1m 22s
16:	learn: 1.5724907	total: 1.42s	remaining: 1m 21s
17:	learn: 1.5546168	total: 1.49s	remaining: 1m 21s
18:	learn: 1.5416895	total: 1.56s	remaining:

160:	learn: 0.8737844	total: 11.7s	remaining: 1m 1s
161:	learn: 0.8720086	total: 11.8s	remaining: 1m 1s
162:	learn: 0.8711773	total: 11.9s	remaining: 1m
163:	learn: 0.8704599	total: 11.9s	remaining: 1m
164:	learn: 0.8686654	total: 12s	remaining: 1m
165:	learn: 0.8665268	total: 12.1s	remaining: 1m
166:	learn: 0.8645938	total: 12.1s	remaining: 1m
167:	learn: 0.8629671	total: 12.2s	remaining: 1m
168:	learn: 0.8618200	total: 12.3s	remaining: 1m
169:	learn: 0.8604794	total: 12.3s	remaining: 1m
170:	learn: 0.8589767	total: 12.4s	remaining: 1m
171:	learn: 0.8570880	total: 12.5s	remaining: 1m
172:	learn: 0.8553519	total: 12.6s	remaining: 1m
173:	learn: 0.8547147	total: 12.6s	remaining: 59.9s
174:	learn: 0.8540803	total: 12.7s	remaining: 59.8s
175:	learn: 0.8528047	total: 12.7s	remaining: 59.7s
176:	learn: 0.8516694	total: 12.8s	remaining: 59.6s
177:	learn: 0.8510833	total: 12.9s	remaining: 59.4s
178:	learn: 0.8493047	total: 12.9s	remaining: 59.3s
179:	learn: 0.8478374	total: 13s	remaining: 59.

320:	learn: 0.7309907	total: 22.1s	remaining: 46.8s
321:	learn: 0.7304663	total: 22.2s	remaining: 46.7s
322:	learn: 0.7300403	total: 22.3s	remaining: 46.7s
323:	learn: 0.7296006	total: 22.3s	remaining: 46.6s
324:	learn: 0.7293581	total: 22.4s	remaining: 46.5s
325:	learn: 0.7290615	total: 22.4s	remaining: 46.4s
326:	learn: 0.7284125	total: 22.5s	remaining: 46.3s
327:	learn: 0.7275690	total: 22.6s	remaining: 46.3s
328:	learn: 0.7266894	total: 22.6s	remaining: 46.2s
329:	learn: 0.7262322	total: 22.7s	remaining: 46.1s
330:	learn: 0.7255446	total: 22.8s	remaining: 46s
331:	learn: 0.7254555	total: 22.8s	remaining: 45.9s
332:	learn: 0.7249450	total: 22.9s	remaining: 45.8s
333:	learn: 0.7246034	total: 22.9s	remaining: 45.7s
334:	learn: 0.7242368	total: 23s	remaining: 45.7s
335:	learn: 0.7240736	total: 23.1s	remaining: 45.6s
336:	learn: 0.7235183	total: 23.1s	remaining: 45.5s
337:	learn: 0.7231713	total: 23.2s	remaining: 45.4s
338:	learn: 0.7227362	total: 23.2s	remaining: 45.3s
339:	learn: 0.72

481:	learn: 0.6677057	total: 32.2s	remaining: 34.6s
482:	learn: 0.6675260	total: 32.2s	remaining: 34.5s
483:	learn: 0.6672308	total: 32.3s	remaining: 34.4s
484:	learn: 0.6667160	total: 32.3s	remaining: 34.3s
485:	learn: 0.6665267	total: 32.4s	remaining: 34.3s
486:	learn: 0.6662498	total: 32.5s	remaining: 34.2s
487:	learn: 0.6658608	total: 32.5s	remaining: 34.1s
488:	learn: 0.6654586	total: 32.6s	remaining: 34.1s
489:	learn: 0.6653323	total: 32.6s	remaining: 34s
490:	learn: 0.6651565	total: 32.7s	remaining: 33.9s
491:	learn: 0.6650938	total: 32.8s	remaining: 33.8s
492:	learn: 0.6648777	total: 32.8s	remaining: 33.8s
493:	learn: 0.6648192	total: 32.9s	remaining: 33.7s
494:	learn: 0.6642167	total: 32.9s	remaining: 33.6s
495:	learn: 0.6639508	total: 33s	remaining: 33.5s
496:	learn: 0.6632399	total: 33.1s	remaining: 33.5s
497:	learn: 0.6630390	total: 33.1s	remaining: 33.4s
498:	learn: 0.6629598	total: 33.2s	remaining: 33.3s
499:	learn: 0.6627152	total: 33.2s	remaining: 33.2s
500:	learn: 0.66

642:	learn: 0.6310373	total: 41.8s	remaining: 23.2s
643:	learn: 0.6310036	total: 41.8s	remaining: 23.1s
644:	learn: 0.6307076	total: 41.9s	remaining: 23.1s
645:	learn: 0.6306234	total: 42s	remaining: 23s
646:	learn: 0.6303332	total: 42s	remaining: 22.9s
647:	learn: 0.6298254	total: 42.1s	remaining: 22.9s
648:	learn: 0.6296222	total: 42.2s	remaining: 22.8s
649:	learn: 0.6294670	total: 42.2s	remaining: 22.7s
650:	learn: 0.6291188	total: 42.3s	remaining: 22.7s
651:	learn: 0.6290036	total: 42.3s	remaining: 22.6s
652:	learn: 0.6289156	total: 42.4s	remaining: 22.5s
653:	learn: 0.6287414	total: 42.5s	remaining: 22.5s
654:	learn: 0.6286457	total: 42.5s	remaining: 22.4s
655:	learn: 0.6284526	total: 42.6s	remaining: 22.3s
656:	learn: 0.6280061	total: 42.6s	remaining: 22.3s
657:	learn: 0.6279914	total: 42.7s	remaining: 22.2s
658:	learn: 0.6277179	total: 42.8s	remaining: 22.1s
659:	learn: 0.6273789	total: 42.8s	remaining: 22.1s
660:	learn: 0.6269619	total: 42.9s	remaining: 22s
661:	learn: 0.626725

803:	learn: 0.5998329	total: 51.5s	remaining: 12.5s
804:	learn: 0.5994529	total: 51.5s	remaining: 12.5s
805:	learn: 0.5992141	total: 51.6s	remaining: 12.4s
806:	learn: 0.5990312	total: 51.7s	remaining: 12.4s
807:	learn: 0.5989668	total: 51.7s	remaining: 12.3s
808:	learn: 0.5989580	total: 51.8s	remaining: 12.2s
809:	learn: 0.5985587	total: 51.9s	remaining: 12.2s
810:	learn: 0.5984062	total: 51.9s	remaining: 12.1s
811:	learn: 0.5983915	total: 52s	remaining: 12s
812:	learn: 0.5983412	total: 52s	remaining: 12s
813:	learn: 0.5981661	total: 52.1s	remaining: 11.9s
814:	learn: 0.5981192	total: 52.1s	remaining: 11.8s
815:	learn: 0.5979273	total: 52.2s	remaining: 11.8s
816:	learn: 0.5976379	total: 52.3s	remaining: 11.7s
817:	learn: 0.5973805	total: 52.3s	remaining: 11.6s
818:	learn: 0.5972292	total: 52.4s	remaining: 11.6s
819:	learn: 0.5971789	total: 52.5s	remaining: 11.5s
820:	learn: 0.5968270	total: 52.5s	remaining: 11.5s
821:	learn: 0.5967907	total: 52.6s	remaining: 11.4s
822:	learn: 0.596625

964:	learn: 0.5736432	total: 1m 1s	remaining: 2.22s
965:	learn: 0.5735995	total: 1m 1s	remaining: 2.15s
966:	learn: 0.5734641	total: 1m 1s	remaining: 2.09s
967:	learn: 0.5734083	total: 1m 1s	remaining: 2.03s
968:	learn: 0.5732476	total: 1m 1s	remaining: 1.96s
969:	learn: 0.5731920	total: 1m 1s	remaining: 1.9s
970:	learn: 0.5731811	total: 1m 1s	remaining: 1.84s
971:	learn: 0.5730497	total: 1m 1s	remaining: 1.77s
972:	learn: 0.5729187	total: 1m 1s	remaining: 1.71s
973:	learn: 0.5726640	total: 1m 1s	remaining: 1.65s
974:	learn: 0.5723273	total: 1m 1s	remaining: 1.58s
975:	learn: 0.5721744	total: 1m 1s	remaining: 1.52s
976:	learn: 0.5721210	total: 1m 1s	remaining: 1.46s
977:	learn: 0.5720676	total: 1m 1s	remaining: 1.39s
978:	learn: 0.5718049	total: 1m 1s	remaining: 1.33s
979:	learn: 0.5717230	total: 1m 2s	remaining: 1.27s
980:	learn: 0.5717116	total: 1m 2s	remaining: 1.2s
981:	learn: 0.5714518	total: 1m 2s	remaining: 1.14s
982:	learn: 0.5712412	total: 1m 2s	remaining: 1.08s
983:	learn: 0.

In [92]:
pred_train = cb.predict(X_train)
pred_test = cb.predict(X_test)

In [93]:
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.8418309942559365
0.7836666666666666


In [88]:
cross_val_score(cb, X, y, cv=5, scoring='accuracy').mean()

Learning rate set to 0.131472
0:	learn: 2.4997238	total: 90.1ms	remaining: 1m 30s
1:	learn: 2.3099386	total: 186ms	remaining: 1m 32s
2:	learn: 2.1921773	total: 277ms	remaining: 1m 32s
3:	learn: 2.0954230	total: 360ms	remaining: 1m 29s
4:	learn: 2.0243749	total: 448ms	remaining: 1m 29s
5:	learn: 1.9544952	total: 537ms	remaining: 1m 29s
6:	learn: 1.8934465	total: 623ms	remaining: 1m 28s
7:	learn: 1.8484767	total: 701ms	remaining: 1m 26s
8:	learn: 1.8042613	total: 768ms	remaining: 1m 24s
9:	learn: 1.7592191	total: 851ms	remaining: 1m 24s
10:	learn: 1.7242214	total: 926ms	remaining: 1m 23s
11:	learn: 1.6930539	total: 1s	remaining: 1m 22s
12:	learn: 1.6623639	total: 1.09s	remaining: 1m 22s
13:	learn: 1.6347050	total: 1.18s	remaining: 1m 23s
14:	learn: 1.6175088	total: 1.25s	remaining: 1m 21s
15:	learn: 1.5981899	total: 1.32s	remaining: 1m 21s
16:	learn: 1.5821978	total: 1.39s	remaining: 1m 20s
17:	learn: 1.5694529	total: 1.46s	remaining: 1m 19s
18:	learn: 1.5463626	total: 1.54s	remaining: 1

160:	learn: 0.8954581	total: 11.3s	remaining: 58.8s
161:	learn: 0.8934685	total: 11.3s	remaining: 58.7s
162:	learn: 0.8925562	total: 11.4s	remaining: 58.6s
163:	learn: 0.8916310	total: 11.5s	remaining: 58.5s
164:	learn: 0.8908341	total: 11.5s	remaining: 58.3s
165:	learn: 0.8894724	total: 11.6s	remaining: 58.2s
166:	learn: 0.8879104	total: 11.7s	remaining: 58.2s
167:	learn: 0.8868082	total: 11.7s	remaining: 58s
168:	learn: 0.8853266	total: 11.8s	remaining: 57.9s
169:	learn: 0.8844137	total: 11.8s	remaining: 57.8s
170:	learn: 0.8826325	total: 11.9s	remaining: 57.7s
171:	learn: 0.8815216	total: 12s	remaining: 57.6s
172:	learn: 0.8797717	total: 12s	remaining: 57.5s
173:	learn: 0.8784515	total: 12.1s	remaining: 57.4s
174:	learn: 0.8775890	total: 12.2s	remaining: 57.3s
175:	learn: 0.8762640	total: 12.2s	remaining: 57.2s
176:	learn: 0.8751394	total: 12.3s	remaining: 57.1s
177:	learn: 0.8742945	total: 12.3s	remaining: 57s
178:	learn: 0.8729860	total: 12.4s	remaining: 56.9s
179:	learn: 0.871942

320:	learn: 0.7330549	total: 21.6s	remaining: 45.6s
321:	learn: 0.7325804	total: 21.6s	remaining: 45.5s
322:	learn: 0.7322865	total: 21.7s	remaining: 45.5s
323:	learn: 0.7317748	total: 21.8s	remaining: 45.4s
324:	learn: 0.7315316	total: 21.8s	remaining: 45.3s
325:	learn: 0.7312747	total: 21.9s	remaining: 45.2s
326:	learn: 0.7305827	total: 21.9s	remaining: 45.1s
327:	learn: 0.7301116	total: 22s	remaining: 45s
328:	learn: 0.7297794	total: 22s	remaining: 45s
329:	learn: 0.7286601	total: 22.1s	remaining: 44.9s
330:	learn: 0.7283204	total: 22.2s	remaining: 44.8s
331:	learn: 0.7278222	total: 22.2s	remaining: 44.7s
332:	learn: 0.7273652	total: 22.3s	remaining: 44.7s
333:	learn: 0.7267305	total: 22.4s	remaining: 44.6s
334:	learn: 0.7266874	total: 22.4s	remaining: 44.5s
335:	learn: 0.7263917	total: 22.5s	remaining: 44.4s
336:	learn: 0.7261356	total: 22.5s	remaining: 44.3s
337:	learn: 0.7254852	total: 22.6s	remaining: 44.2s
338:	learn: 0.7250483	total: 22.6s	remaining: 44.2s
339:	learn: 0.724700

481:	learn: 0.6698905	total: 31.2s	remaining: 33.5s
482:	learn: 0.6695376	total: 31.2s	remaining: 33.4s
483:	learn: 0.6691370	total: 31.3s	remaining: 33.4s
484:	learn: 0.6687950	total: 31.4s	remaining: 33.3s
485:	learn: 0.6686884	total: 31.4s	remaining: 33.2s
486:	learn: 0.6680999	total: 31.5s	remaining: 33.2s
487:	learn: 0.6678702	total: 31.5s	remaining: 33.1s
488:	learn: 0.6677551	total: 31.6s	remaining: 33s
489:	learn: 0.6675129	total: 31.7s	remaining: 32.9s
490:	learn: 0.6671055	total: 31.7s	remaining: 32.9s
491:	learn: 0.6669712	total: 31.8s	remaining: 32.8s
492:	learn: 0.6665841	total: 31.8s	remaining: 32.7s
493:	learn: 0.6664548	total: 31.9s	remaining: 32.7s
494:	learn: 0.6662815	total: 31.9s	remaining: 32.6s
495:	learn: 0.6660684	total: 32s	remaining: 32.5s
496:	learn: 0.6660503	total: 32s	remaining: 32.4s
497:	learn: 0.6655996	total: 32.1s	remaining: 32.4s
498:	learn: 0.6651056	total: 32.2s	remaining: 32.3s
499:	learn: 0.6647630	total: 32.2s	remaining: 32.2s
500:	learn: 0.6644

640:	learn: 0.6312825	total: 40.4s	remaining: 22.6s
641:	learn: 0.6309379	total: 40.5s	remaining: 22.6s
642:	learn: 0.6305505	total: 40.5s	remaining: 22.5s
643:	learn: 0.6300092	total: 40.6s	remaining: 22.4s
644:	learn: 0.6297349	total: 40.7s	remaining: 22.4s
645:	learn: 0.6293523	total: 40.7s	remaining: 22.3s
646:	learn: 0.6288569	total: 40.8s	remaining: 22.3s
647:	learn: 0.6287923	total: 40.8s	remaining: 22.2s
648:	learn: 0.6287271	total: 40.9s	remaining: 22.1s
649:	learn: 0.6284848	total: 41s	remaining: 22.1s
650:	learn: 0.6284665	total: 41s	remaining: 22s
651:	learn: 0.6280559	total: 41.1s	remaining: 21.9s
652:	learn: 0.6277976	total: 41.1s	remaining: 21.9s
653:	learn: 0.6275552	total: 41.2s	remaining: 21.8s
654:	learn: 0.6271211	total: 41.3s	remaining: 21.7s
655:	learn: 0.6270545	total: 41.3s	remaining: 21.7s
656:	learn: 0.6267926	total: 41.4s	remaining: 21.6s
657:	learn: 0.6266284	total: 41.4s	remaining: 21.5s
658:	learn: 0.6264041	total: 41.5s	remaining: 21.5s
659:	learn: 0.6261

...

497:	learn: 0.6611660	total: 32.3s	remaining: 32.5s
498:	learn: 0.6607184	total: 32.4s	remaining: 32.5s
499:	learn: 0.6601666	total: 32.4s	remaining: 32.4s
500:	learn: 0.6599848	total: 32.5s	remaining: 32.3s
501:	learn: 0.6598010	total: 32.5s	remaining: 32.3s
502:	learn: 0.6597786	total: 32.6s	remaining: 32.2s
503:	learn: 0.6595257	total: 32.6s	remaining: 32.1s
504:	learn: 0.6587734	total: 32.7s	remaining: 32.1s
505:	learn: 0.6586720	total: 32.8s	remaining: 32s
506:	learn: 0.6583788	total: 32.8s	remaining: 31.9s
507:	learn: 0.6581573	total: 32.9s	remaining: 31.8s
508:	learn: 0.6579108	total: 32.9s	remaining: 31.8s
509:	learn: 0.6575370	total: 33s	remaining: 31.7s
510:	learn: 0.6571366	total: 33.1s	remaining: 31.6s
511:	learn: 0.6571204	total: 33.1s	remaining: 31.6s
512:	learn: 0.6570886	total: 33.2s	remaining: 31.5s
513:	learn: 0.6565016	total: 33.2s	remaining: 31.4s
514:	learn: 0.6564656	total: 33.3s	remaining: 31.3s
515:	learn: 0.6564443	total: 33.3s	remaining: 31.3s
516:	learn: 0.65

659:	learn: 0.6215186	total: 41.9s	remaining: 21.6s
660:	learn: 0.6214469	total: 42s	remaining: 21.5s
661:	learn: 0.6211223	total: 42s	remaining: 21.5s
662:	learn: 0.6207724	total: 42.1s	remaining: 21.4s
663:	learn: 0.6204339	total: 42.2s	remaining: 21.3s
664:	learn: 0.6199949	total: 42.2s	remaining: 21.3s
665:	learn: 0.6192628	total: 42.3s	remaining: 21.2s
666:	learn: 0.6189663	total: 42.4s	remaining: 21.1s
667:	learn: 0.6185210	total: 42.4s	remaining: 21.1s
668:	learn: 0.6184142	total: 42.5s	remaining: 21s
669:	learn: 0.6181243	total: 42.6s	remaining: 21s
670:	learn: 0.6180781	total: 42.6s	remaining: 20.9s
671:	learn: 0.6180153	total: 42.7s	remaining: 20.8s
672:	learn: 0.6179181	total: 42.7s	remaining: 20.8s
673:	learn: 0.6178166	total: 42.8s	remaining: 20.7s
674:	learn: 0.6177162	total: 42.8s	remaining: 20.6s
675:	learn: 0.6175061	total: 42.9s	remaining: 20.6s
676:	learn: 0.6174258	total: 42.9s	remaining: 20.5s
677:	learn: 0.6173270	total: 43s	remaining: 20.4s
678:	learn: 0.6172708	

819:	learn: 0.5945808	total: 51.3s	remaining: 11.3s
820:	learn: 0.5944707	total: 51.4s	remaining: 11.2s
821:	learn: 0.5943788	total: 51.4s	remaining: 11.1s
822:	learn: 0.5940244	total: 51.5s	remaining: 11.1s
823:	learn: 0.5939093	total: 51.5s	remaining: 11s
824:	learn: 0.5939026	total: 51.6s	remaining: 10.9s
825:	learn: 0.5938042	total: 51.7s	remaining: 10.9s
826:	learn: 0.5936190	total: 51.7s	remaining: 10.8s
827:	learn: 0.5935803	total: 51.8s	remaining: 10.8s
828:	learn: 0.5933731	total: 51.8s	remaining: 10.7s
829:	learn: 0.5932238	total: 51.9s	remaining: 10.6s
830:	learn: 0.5926676	total: 52s	remaining: 10.6s
831:	learn: 0.5926319	total: 52s	remaining: 10.5s
832:	learn: 0.5924118	total: 52.1s	remaining: 10.4s
833:	learn: 0.5921726	total: 52.2s	remaining: 10.4s
834:	learn: 0.5920813	total: 52.2s	remaining: 10.3s
835:	learn: 0.5919818	total: 52.3s	remaining: 10.3s
836:	learn: 0.5919120	total: 52.3s	remaining: 10.2s
837:	learn: 0.5915928	total: 52.4s	remaining: 10.1s
838:	learn: 0.5915

981:	learn: 0.5696297	total: 1m	remaining: 1.11s
982:	learn: 0.5694291	total: 1m	remaining: 1.05s
983:	learn: 0.5693811	total: 1m	remaining: 991ms
984:	learn: 0.5692453	total: 1m	remaining: 929ms
985:	learn: 0.5689489	total: 1m 1s	remaining: 867ms
986:	learn: 0.5687698	total: 1m 1s	remaining: 805ms
987:	learn: 0.5685859	total: 1m 1s	remaining: 743ms
988:	learn: 0.5685774	total: 1m 1s	remaining: 681ms
989:	learn: 0.5684517	total: 1m 1s	remaining: 619ms
990:	learn: 0.5682723	total: 1m 1s	remaining: 557ms
991:	learn: 0.5681348	total: 1m 1s	remaining: 495ms
992:	learn: 0.5679540	total: 1m 1s	remaining: 433ms
993:	learn: 0.5679136	total: 1m 1s	remaining: 371ms
994:	learn: 0.5678962	total: 1m 1s	remaining: 309ms
995:	learn: 0.5675974	total: 1m 1s	remaining: 247ms
996:	learn: 0.5674137	total: 1m 1s	remaining: 186ms
997:	learn: 0.5674039	total: 1m 1s	remaining: 124ms
998:	learn: 0.5673931	total: 1m 1s	remaining: 61.9ms
999:	learn: 0.5671973	total: 1m 1s	remaining: 0us


0.7845326403058948

Cross validation score: 0.7845326403058948

### Kaggle submission

In [94]:
pred_kaggle = cb.predict(X_kaggle)
pred_kaggle

array([['british'],
       ['southern_us'],
       ['italian'],
       ...,
       ['italian'],
       ['southern_us'],
       ['mexican']], dtype=object)

In [95]:
submission = pd.DataFrame(pred_kaggle, index=test.index, columns=['cuisine'])
submission

,cuisine
id,
18009,british
28583,southern_us
41580,italian
29752,cajun_creole
35687,italian
...,...
30246,french
36028,southern_us
22339,italian


In [96]:
submission.to_csv('submission3.csv')

Kaggle score: 0.78318